# Manual approval for Prompt Management in Amazon Bedrock

Amazon Bedrock allows you to create and save prompts in a Prompt management. A prompt is a reusable message that can be integrated with larger applications. You can create your own prompts, select models to run inference on them, and configure the inference parameters to use.

You can continually test your prompt and iterate on it. When you're satisfied with a configuration, you can then create a version of it to deploy to production. To ensure quality and reliability, it is recommended to implement a manual approval process for new versions of prompts before deploying them to production environments.

In this notebook we will learn how to build a manual approval workflow for prompt management.

## Import needed libraries and clients
Let's start by importing the needed libraries and setting up the clients needed for the notebook to work. 

In [ ]:
import json
import time
import uuid
import boto3
from datetime import datetime
from src.utils import create_base_infrastructure, create_bedrock_flow_role, create_dynamodb_item, prepare_and_create_flow_alias, executePromptFlow, update_flow_prompt
dynamodb_resource = boto3.resource('dynamodb')

iam = boto3.client('iam')
sns = boto3.client('sns')
bedrock_ag = boto3.client('bedrock-agent')
cloudformation = boto3.client('cloudformation')
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')

## Create the infrastructure for the solution 
We are going to deploy the infrastructure for this solution using an AWS CloudFormation template we have already created. 

The template will deploy the following:

**SNS topic**: A publish-subscribe messaging service for sending notifications about new prompt versions requiring approval.

**API Gateway**: A fully managed service for creating, publishing, and securing APIs, used for exposing the approve and reject endpoints.

**DynamoDB table**: A NoSQL database for storing prompt metadata, including the prompt text, version, and approval status.

**Lambda Functions**:
- `TriggerLambdaFunction`: A serverless function triggered by DynamoDB streams to send approval notifications via SNS.
    
- `ApproveLambdaFunction`: A serverless function invoked by the API Gateway to update the prompt version status to "Approved" in DynamoDB.
    
- `RejectLambdaFunction`: A serverless function invoked by the API Gateway to update the prompt version status to "Rejected" in DynamoDB.

In [ ]:
def short_uuid():
    uuid_str = str(uuid.uuid4())
    return uuid_str[:4]

solution_id = 'pmma{}'.format(short_uuid()).lower()
dynamodb_table_name, sns_topic_arn = create_base_infrastructure(solution_id)

## Subscribe an approvers' email to the SNS Topic
We are going to subscribe an email address to the SNS topic

In [ ]:
email_address = '<INSERT YOUR EMAIL>'
try:
    response = sns.subscribe(
        TopicArn=sns_topic_arn,
        Protocol='email',
        Endpoint=email_address
    )
    print(f"Subscription created: {response['SubscriptionArn']}")
except Exception as e:
    print(f"Error subscribing email: {e}")

#time.sleep(20)

 <div class="alert alert-block alert-warning">
    <b>IMPORTANT</b>: You will need to accept the subscription email to receive actions emails. The confirmation email can take a couple of minutes to arrive.
 </div>


## Add a new prompt to Prompt Management
We are going to create a base prompt in Amazon Bedrock Prompt Management and Amazon DynamoDB.

In [ ]:
promptText ="""
                You're an assistant which extracts data from text. Consider the input text in the <input_text> tags and extract the following information: \
                Name, City, Company. Return this data in a json format to later on process.
                
                <input_text>
                    {{input_text}}
                </input_text>
    
                Skip any preamble or any other text apart from the JSON in your answer.
                
                """

In [ ]:
modelInvokeId = "amazon.titan-text-premier-v1:0"
prompt_name = "data-extraction-prompt-{}".format(solution_id)

response = bedrock_ag.create_prompt(
    name = prompt_name,
    description = "Prompt for extracting a set of entities from a provided text.",
    variants = [
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 2000,
                    "temperature": 0,
                }
            },
            "modelId": modelInvokeId,
            "name": "variantOne",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input_text"
                        },
                        {
                            "name": "output"
                        }
                    ],
                    "text": promptText
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variantOne"
)

In [ ]:
promptId = response["id"]
promptArn = response["arn"]
promptName = response["name"]
promptVersion = response["version"]
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}\nPrompt Version: {promptVersion}")

### Store the information in Amazon DynamoDB
Let's store this information in our Amazon DynamoDB table.

In [ ]:
print(create_dynamodb_item(dynamodb_table_name, promptId, promptName, promptVersion, promptText))

## Approve or Deny the Prompt
Once the prompt has been submitted, you will receive an email with the two options to take, Approve or Deny.

Each action will do it's purpose and change the value in the Amazon DynamoDB table. 

Head over to the Amazon DynamoDB table to see the status change.

## Create Prompt Flow
We will use Amazon Bedrock Prompt Flows to execute our prompt. 

In [ ]:
flow_name = "prompt-code-example-{}".format(solution_id)
flow_description = "This is an example flow"
flow_alias_description = "Alias for my prompt flow"

print("Starting the flow creation process...")
flow_role_arn = create_bedrock_flow_role("example-flow-role-{}".format(solution_id))
flow_id, flow_alias_id = prepare_and_create_flow_alias(flow_name, flow_description, flow_role_arn, promptArn, flow_alias_description)

In [ ]:
executePromptFlow("My name is Dani Mitchell and I live in Madrid and work at AWS.", flow_id, flow_alias_id)

## Create a new prompt version
When we create a new prompt version, the prompt management administrator will receive a notification regarding the new version created to approve it or not.

### Update the prompt

In [ ]:
updatedPromptText ="""
                    You're an assistant which extracts data from text. Consider the input text in the <input_text> tags and extract the following information: \
                    Name, Surname, City, Company. Return this data in a json format to later on process.
                    
                    <input_text>
                        {{input_text}}
                    </input_text>
        
                    Skip any preamble or any other text apart from the JSON in your answer.
                
                """

response = bedrock_ag.update_prompt(
    name=promptName,
    promptIdentifier= promptId,
    description = "Prompt for extracting a set of entities from a provided text.",
    variants = [
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 2000,
                    "temperature": 0,
                }
            },
            "modelId": modelInvokeId,
            "name": "variantOne",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input_text"
                        },
                        {
                            "name": "output"
                        }
                    ],
                    "text": updatedPromptText
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variantOne"
)

### Create a prompt version

In [ ]:
response = bedrock_ag.create_prompt_version(
    description='Included "Surname" as additional entity to extract',
    promptIdentifier=promptId
)

In [ ]:
promptId = response['id']
promptName = response['name']
promptVersion = response['version']
promptText = response['variants'][0]['templateConfiguration']['text']['text']
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}\nPrompt Version: {promptVersion}")

In [ ]:
create_dynamodb_item(dynamodb_table_name, promptId, promptName, promptVersion, promptText)

## Approve or Deny the Prompt Version Update
Once the new version has been submitted, you will receive an email with the two options to take, Approve or Deny.

Each action will do it's purpose and change the value in the Amazon DynamoDB table. 

Head over to the Amazon DynamoDB table to see the status change.

## Update the Prompt Flow and create a new version

In [ ]:
update_flow_prompt(flow_id, promptArn, promptId, promptVersion, flow_name, flow_description, flow_role_arn, dynamodb_table_name, flow_alias_id, flow_alias_description)

In [ ]:
executePromptFlow("My name is Dani Mitchell and I live in Madrid and work at AWS.", flow_id, flow_alias_id)

## Delete resources

Run the following cell to delete the created resources and avoid unnecesary costs. This should take about 1 minute to complete.

In [ ]:
try:
    # Retrieve the stack information
    stack_info = cloudformation.describe_stacks(StackName=solution_id)
    stack_status = stack_info['Stacks'][0]['StackStatus']

    # Check if the stack exists and is in a deletable state
    if stack_status != 'DELETE_COMPLETE':
        # Delete the stack
        cloudformation.delete_stack(StackName=solution_id)
        print(f'Deleting stack: {solution_id}')

        # Wait for the stack deletion to complete
        waiter = cloudformation.get_waiter('stack_delete_complete')
        waiter.wait(StackName=solution_id)
        print(f'Stack {solution_id} deleted successfully.')
    else:
        print(f'Stack {solution_id} does not exist or has already been deleted.')

except cloudformation.exceptions.ClientError as e:
    print(f'Error deleting stack {solution_id}: {e.response["Error"]["Message"]}')

# Delete the prompt
try:
    response = bedrock_ag.delete_prompt(
        promptIdentifier=promptId
    )
    print(f'Prompt {promptId} deleted successfully.')
except Exception as e:
    print(f'Error deleting prompt {promptId}: {e}')

# Delete the flow alias
try:
    response = bedrock_ag.delete_flow_alias(
        aliasIdentifier=flow_alias_id,
        flowIdentifier=flow_id
    )
    print(f'Flow alias deleted successfully.')
except Exception as e:
    print(f'Error deleting flow: {e}')

# Delete the flow
try:
    response = bedrock_ag.delete_flow(
        flowIdentifier=flow_id,
        skipResourceInUseCheck=False
    )
    print(f'Flow deleted successfully.')
except Exception as e:
    print(f'Error deleting flow: {e}')